In [1]:
#!/usr/bin/env python import nntplib
import nntplib
import socket

# HOST = 'your.nntp.server'
GRNM = 'comp.lang.python'
USER = 'wesley'
PASS = 'youllNeverGuess'

HOST = 'web.aioe.org' # 网上找了个能用的

def main():

    try:
        n = nntplib.NNTP(HOST)  # , user=USER, password=PASS) #尝试连接 NNTP 服务器，如果失败就退出
    except socket.gaierror as e:
        print('ERROR: cannot reach host "%s"' % HOST)
        print(' ("%s")' % eval(str(e))[1])
        return
    except nntplib.NNTPPermanentError as e:
        print('ERROR: access denied on "%s"' % HOST)
        print(' ("%s")' % str(e))
        return
    print('*** Connected to host "%s"' % HOST)
    try:
        rsp, ct, fst, lst, grp = n.group(GRNM) #尝试读取指定的新闻组
    except nntplib.NNTPTemporaryError as e:
        print('ERROR: cannot load group "%s"' % GRNM)
        print('("%s")' % str(e))
        print(' Server may require authentication')
        print("Uncomment/edit login line above")
        return
    except nntplib.NNTPTemporaryError as e:
        print('ERROR: group "%s" unavailable' % GRNM)
        print(' ("%s")' % str(e))
        n.quit()
        return
    print('*** Found newsgroup "%s"' % GRNM)

    rng = '%s-%s' % (lst, lst) #给定想要提取消息头的文章范围，因为要获取最后一条信息，所以范围是最后--最后
    rsp, frm = n.xhdr('from', rng)
    rsp, sub = n.xhdr('subject', rng)
    rsp, dat = n.xhdr('date', rng)
    print('''*** Found last article (#%s):
            From: %s
            Subject: %s
            Date: %s
        '''% (lst, frm[0][1], sub[0][1], dat[0][1]))

    #根据最后一篇文章的 id 获取（服务器响应信息，文章编号，消息 id，文章所有行）
    rsp, (anum, mid, data) = n.body(lst)  # return resp, ArticleInfo(art_num, message_id, lines)
    
    displayFirst20(data)
    n.quit()

def displayFirst20(data):
    print('*** First (<= 20) meaningful lines:\n')
    count = 0

    lines = (line.rstrip() for line in data)
    lastBlank = True
    for line in lines:
        if line:
            # lower = line.lower() 后面lower.startswith('>')报错; 数据类型为byte
            lower = str(line.lower(),encoding='utf-8')    
            
            if (lower.startswith('>') and not
                    lower.startswith('>>>')) or \
                    lower.startswith('|') or  \
                    lower.startswith('in article') or  \
                    lower.endswith('writes:') or  \
                    lower.endswith('wrote:'):
                    continue
        if not lastBlank or (lastBlank and line):
            print(' %s' % line)
            if line:
                count += 1
                lastBlank = False
            else:
                lastBlank = True
        if count == 20:
            break

if __name__ == '__main__':
    main()




*** Connected to host "web.aioe.org"
*** Found newsgroup "comp.lang.python"
*** Found last article (#221883):
            From: Paul Rubin <no.email@nospam.invalid>
            Subject: Re: Amber Brown: Batteries Included, But They're Leaking
            Date: Sun, 19 May 2019 00:09:24 -0700
        
*** First (<= 20) meaningful lines:

 b'There is a Hacker News thread with more discussion of this issue:'
 b''
 b'https://news.ycombinator.com/item?id=19948642'
 b''
 b'I still think batteries included is a good aspirational goal, but maybe'
 b"it's harder to do now than it was 20 years ago, because there is so much"
 b'more code around.'
